<a href="https://colab.research.google.com/github/aminghafari1/lcaging-fmri/blob/main/Copy_of_stimulus_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## This code is for reading the subject data and trying to get the EV files responsible for the visual and auditory stimuli.
## The outline is that the subject number is entered and the resulting output is EV files for all the contrasts tha are considered.
## For now I'm gonna only focus on separating the effect of the grip from the V/A stimuli, then we will see what to do with the 
## different stimulus/handgrip levels and also the subject answers and the confidence ratings.

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import glob
import os
def substring_after(s,delim):
  return s.partition(delim)[2]
import scipy.io as sio


Mounted at /content/gdrive


In [ ]:
sub_num=input("Please enter the subject number:")
print("Working on the visual and auditory data of sub %s now." %(sub_num))

base_dir=glob.glob('/content/gdrive/Shareddrives/LC-Aging/'\
  'Older Adult - MRI Study/Data Collection/BAP/BAP data/')
base_dir=base_dir[0]
sub_folder_name="sub-BAP"+sub_num
z=os.listdir(base_dir)
print("The folder we are looking for here is: %s" %(sub_folder_name))
while sub_num != "quit":
    if sub_folder_name in z:
        print("The folder %s was found in the list." %(sub_folder_name) )
        break
    else:
        print("The folder %s was not found in the list." %(sub_folder_name) )
        print("Please enter another subject code or type 'quit' to exit.")
    sub_num = input("Please enter a subject number or type 'quit' to exit: ")
subject_folder=base_dir+sub_folder_name
print(subject_folder)
sessions=[2,3]
for session_number in sessions:
  temp="/ses-%s/InsideScanner" %(session_number)
  scans_folder=subject_folder+temp
  scans_contents=os.listdir(scans_folder)
  mat_files= [element for element in scans_contents if element.endswith('.mat') and "eyetrack" not in element and "MVCnPRAC" not in element]
  temp_perf=mat_files[0]
  if "Voddball" in temp_perf:
    visual_files=mat_files
    visual_scans_folder=scans_folder+"/"
  else:
    audio_files=mat_files
    audio_scans_folder=scans_folder+"/"
  
## Finding the save directory
res_directory="BAP"+str(sub_num)
print(res_directory)
save_dir_vis='/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/visual' %(res_directory)
save_dir_aud='/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/%s/auditory' %(res_directory)


Please enter the subject number:103
Working on the visual and auditory data of sub 103 now.
The folder we are looking for here is: sub-BAP103
The folder sub-BAP103 was found in the list.
/content/gdrive/Shareddrives/LC-Aging/Older Adult - MRI Study/Data Collection/BAP/BAP data/sub-BAP103
BAP103


In [ ]:
"BAP"+str(sub_num)

'BAP103'

In [ ]:
## Because in some of the subjects the number of auditory tasks might differ from the visual ones
## we need to exploit 2 for loops, one for each here.
## This section is only for the visual test
number_of_trials_per_run=30
visual_ones=np.ones(2*number_of_trials_per_run)
grip_ones=np.ones(number_of_trials_per_run)
for current_file in visual_files:
  task_file=current_file
  file_directory=visual_scans_folder+task_file
  temp=substring_after(file_directory,"run")
  run_index=temp[0]
  print("Now reading run number %s of the visual test." %(run_index))
  current_run=sio.loadmat(file_directory)
  experiment_start_time=current_run['ExpStartTimeP']
  trial_start=current_run['stimulusStartTimeP']-experiment_start_time ##This is also grip start time
  grip_end_time=current_run['relaxStartTimeP']-experiment_start_time
  first_stimulus_start=current_run['SoundStartTimeP']-experiment_start_time
  first_stimulus_end=current_run['g1_OffsetTimeP']-experiment_start_time
  second_stimulus_end=current_run['g2_OffsetTimeP']-experiment_start_time
  second_stimulus_start=current_run['g2_OnsetTimeP']-experiment_start_time
  grip_length=grip_end_time-trial_start
  first_stimulus_length=first_stimulus_end-first_stimulus_start
  second_stimulus_length=second_stimulus_end-second_stimulus_start
  visual_first_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  visual_second_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  grip_first_vector=np.zeros(np.shape(trial_start)[1])
  grip_second_vector=np.zeros(np.shape(trial_start)[1])
  j=0
  for i in range(np.shape(first_stimulus_start)[1]):
    grip_first_vector[i]=trial_start[0,i]
    grip_second_vector[i]=grip_length[0,i]
    if j%2 ==0:
      visual_first_vector[j]=first_stimulus_start[0,i]
      visual_second_vector[j]=first_stimulus_length[0,i]
      j=j+1
    if j%2 !=0:
      visual_first_vector[j]=second_stimulus_start[0,i]
      visual_second_vector[j]=second_stimulus_length[0,i]
      j=j+1
  visual_ev=np.column_stack((visual_first_vector,visual_second_vector,visual_ones)) 
  grip_ev=np.column_stack((grip_first_vector,grip_second_vector,grip_ones)) 
  visual_filename=f"visual_run_{run_index}.txt"
  ev1_adress=save_dir_vis+visual_filename
  grip_filename=f"visual_grip_run_{run_index}.txt"
  ev2_adress=save_dir_vis+grip_filename
  fmt = ['%.4f', '%.4f', '%d']
  np.savetxt(ev1_adress,visual_ev, delimiter=" ",fmt=fmt)
  np.savetxt(ev2_adress,grip_ev, delimiter=" ",fmt=fmt)




Now reading run number 1 of the visual test.
Now reading run number 2 of the visual test.
Now reading run number 3 of the visual test.
Now reading run number 4 of the visual test.
Now reading run number 5 of the visual test.


In [ ]:
ev_file_adress=save_dir_vis+grip_filename
ev_file_adress

'/content/gdrive/Shareddrives/LC-Aging/LC_aging_fMRI/BAP102/visual_grip_run_5.txt'

In [ ]:
## Because in some of the subjects the number of auditory tasks might differ from the visual ones
## we need to exploit 2 for loops, one for each here.
## This section is only for the auditory test
number_of_trials_per_run=30
audio_ones=np.ones(2*number_of_trials_per_run)
grip_ones=np.ones(number_of_trials_per_run)
for current_file in audio_files:
  task_file=current_file
  file_directory=audio_scans_folder+task_file
  temp=substring_after(file_directory,"run")
  run_index=temp[0]
  print("Now reading run number %s of the auditory test." %(run_index))
  current_run=sio.loadmat(file_directory)
  experiment_start_time=current_run['ExpStartTimeP']
  trial_start=current_run['stimulusStartTimeP']-experiment_start_time ##This is also grip start time
  grip_end_time=current_run['relaxStartTimeP']-experiment_start_time
  first_stimulus_start=current_run['SoundStartTimeP']-experiment_start_time
  first_stimulus_end=first_stimulus_start+0.1 ## Seems like we don't have the exact timings
  second_stimulus_start=first_stimulus_end+0.5
  second_stimulus_end=second_stimulus_start+0.1

  grip_length=grip_end_time-trial_start
  first_stimulus_length=first_stimulus_end-first_stimulus_start
  second_stimulus_length=second_stimulus_end-second_stimulus_start
  audio_first_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  audio_second_vector=np.zeros(2*np.shape(first_stimulus_start)[1])
  grip_first_vector=np.zeros(np.shape(trial_start)[1])
  grip_second_vector=np.zeros(np.shape(trial_start)[1])
  j=0
  for i in range(np.shape(first_stimulus_start)[1]):
    grip_first_vector[i]=trial_start[0,i]
    grip_second_vector[i]=grip_length[0,i]
    if j%2 ==0:
      audio_first_vector[j]=first_stimulus_start[0,i]
      audio_second_vector[j]=first_stimulus_length[0,i]
      j=j+1
    if j%2 !=0:
      audio_first_vector[j]=second_stimulus_start[0,i]
      audio_second_vector[j]=second_stimulus_length[0,i]
      j=j+1
  audio_ev=np.column_stack((audio_first_vector,audio_second_vector,audio_ones)) 
  grip_ev=np.column_stack((grip_first_vector,grip_second_vector,grip_ones)) 
  audio_filename=f"audio_run_{run_index}.txt"
  ev1_adress=save_dir_aud+audio_filename
  grip_filename=f"audio_grip_run_{run_index}.txt"
  ev2_adress=save_dir_aud+grip_filename
  fmt = ['%.4f', '%.4f', '%d']
  np.savetxt(ev1_adress,audio_ev, delimiter=" ",fmt=fmt)
  np.savetxt(ev2_adress,grip_ev, delimiter=" ",fmt=fmt)



Now reading run number 1 of the auditory test.
Now reading run number 2 of the auditory test.
Now reading run number 3 of the auditory test.
Now reading run number 4 of the auditory test.
Now reading run number 5 of the auditory test.


In [ ]:
first_stimulus_start

array([[  3.8099293,  16.0206326,  29.8313092,  42.2615242,  56.2621671,
         72.0135609,  87.7638165, 103.5049449, 119.2555947, 133.2561306,
        145.5063155, 159.5073296, 173.5076996, 185.758313 , 198.0091031,
        210.2594204, 224.2603517, 240.0115014, 252.2615588, 264.5121072,
        280.2530545, 294.2535608, 308.2540974, 322.2545207, 336.2554879,
        348.5062499, 362.5065597, 374.7572263, 387.0081127, 401.0084801]])

In [ ]:
visual_files

['subjectBAP103_Voddball_session2_run1_7_6_13_15.mat',
 'subjectBAP103_Voddball_session2_run2_7_6_13_25.mat',
 'subjectBAP103_Voddball_session2_run3_7_6_13_34.mat',
 'subjectBAP103_Voddball_session2_run4_7_6_13_42.mat',
 'subjectBAP103_Voddball_session2_run5_7_6_13_51.mat']

In [ ]:
import os
z=os.listdir(base_dir)
type(z)

list